In [19]:
import sys
import math
import pandas as pd
import numpy as np
sys.path.insert(0, '/Users/beidan/RASHIP/nrde/autoautophrase/')
from activelearning.featureExtraction.hierclusteringscipy import cleandata, normalize
import tools.fileHandler as fh
# import nbimporter
# from activelearning.learning import interface
# dh = interface.datahandler()

In [3]:
class datahandler:
    def __init__(self):
        self.dataset = pd.read_csv('../../outputs/features_results_all_withphrase.csv')
        self.dataset = self.cleandata(self.dataset, 17)
        self.X_normalized = normalize(self.dataset.iloc[:, list(range(1,18))])
        self.phrases = self.dataset.iloc[:, 0].values.tolist()
        self.status = [0] * len(self.X_normalized)

    def cleandata(self, dataset, columnidx):
        f_knownphrase = dataset.iloc[:, [columnidx]].values
        for i in range(len(f_knownphrase)):
            if math.isnan(f_knownphrase[i]):
                f_knownphrase[i] = 0
        dataset.iloc[:, [columnidx]] = f_knownphrase
        return dataset

    def scoreItem(self, phrase, positive=False):
        idx = self.phrases.index(phrase)
        if positive:
           self.status[idx] = 1
        else:
           self.status[idx] = -1
    
    def modifyItems(self, phraselist, positive=False):
        for phrase in phraselist:
            if positive:
                continue
            else:
                self.scoreItem(phrase)
    
    def getAvailableItems(self):
        return [t for idx, t in enumerate(self.phrases) if int(self.status[idx]) == 0]
    
    def getdata(self):
        return self.dataset
    
    def getallidx(self, phrases):
        return [self.phrases.index(t) for t in phrases]

In [4]:
dh = datahandler()
dataset = dh.getdata()
avail = dh.getAvailableItems()


In [5]:
group3 = fh.getwords('../../outputs/features_group3_part_selected.txt', split=False)
group3 = [t for t in group3 if len(t.split(' '))>1]
round = math.ceil(len(group3) / 20)
print("{} round in total.".format(round))

68 round in total.


In [41]:
## get items for training
features_idx = dh.getallidx(group3)
features = (dh.dataset.iloc[features_idx, :])
feature_name_list = list(features.columns)
X = features.values
labels = np.ones((len(X),1))
f_normalized = dh.X_normalized.iloc[features_idx, :]
features.describe().head()

,freq,f_len,max_subgram_freq,partial_order,min_support,mutual_confidence,f_avg_wordlength,f_pos_nn,f_pos_prop,f_pos_verb,f_pos_adj,f_pos_deter,f_pos_empty,f_pos_other,f_completeness,f_postagUniqueness,is_know_phrase
count,1345.000000,1345.000000,1345.000000,1345.000000,1345.000000,1345.0,1345.000000,1345.000000,1345.000000,1345.000000,1345.000000,1345.000000,1345.0,1345.0,1345.0,1345.000000,1345.0
mean,6.198513,2.188848,1.176952,0.005948,0.296446,-1.0,7.726223,0.769283,0.000372,0.050987,0.179172,0.000186,0.0,0.0,0.0,0.079554,4.0
std,29.692296,0.472481,7.824046,0.076922,0.867276,0.0,1.884635,0.245190,0.009637,0.148413,0.237488,0.006817,0.0,0.0,0.0,0.270702,0.0
min,1.000000,2.000000,0.000000,0.000000,0.000000,-1.0,1.500000,0.333000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,4.0
25%,1.000000,2.000000,0.000000,0.000000,0.070000,-1.0,6.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,4.0


In [58]:
# test = fh.getwords('../../outputs/features_group4_part.txt', split=False)
# test_idx = dh.getallidx(test)
# test_normalized = dh.X_normalized.iloc[test_idx, :]
# test_pred = rf.predict(test_normalized)
# test_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [42]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(f_normalized, labels, test_size = 0.2, random_state = 42)

In [43]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1076, 17)
Training Labels Shape: (1076, 1)
Testing Features Shape: (269, 17)
Testing Labels Shape: (269, 1)


In [44]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [53]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', (np.mean(errors)), 'degrees.')


Mean Absolute Error: 0.0 degrees.


In [ ]:
# why positive only?
test = fh.getwords('../../outputs/features_group4_part.txt', split=False)
# test_idx = dh.getallidx(test)
# test_normalized = dh.X_normalized.iloc[test_idx, :]
# test_pred = rf.predict(test_normalized)
# test_pred

In [52]:
np.mean(errors)

0.0

In [ ]:
# interface saved for activelearning
for i in range(round):
    if i < 2:
        items = dh.phrases[i*20 : (i+1)*20]
        items_tousers = [str(idx) + "---" + str(t) for idx, t in enumerate(items)]
        print(items_tousers)
        print("        ============       ")
        choices = input()
        clist = choices.split(',')
        plist = [items[int(t)] for t in clist]
        dh.modifyItems(plist)

